# **Importacion De Librerias y Datos**

In [ ]:
import pandas as pd
import numpy as np

# Definir los tipos de dato para las columnas relevantes en df_salud
dtypes_salud = {
    'Nombre_Tipo_Atencion_Arp': 'category',
    'CODIGO_PROCEDIMIENTO': 'category',
    'Cantidad': 'float64',
    'TIPIFICACION': 'category',
    'DSTIPO_EXAMEN': 'category',
    'Siniestro_Diagnosti_Princi_Id': 'category',
    'Nombre_Municipio_IPS': 'category',
    'Nombre_Departamento_IPS': 'category',
    'Geogra_Municipio_Id': 'category',
    'Nombre_Establecimiento': 'category',
    'Nombre_Municipio_Establecimiento': 'category',
    'Nombre_Departamento_Establecimiento': 'category'

}

# Seleccionar solo las columnas relevantes para df_salud
cols_to_use_salud = [
    'FECHA_ATENCION',              # Fecha y hora del servicio
    'Nombre_Tipo_Atencion_Arp',      # Tipo de atención
    'CODIGO_PROCEDIMIENTO',         # Código del procedimientod
    'Cantidad',                    # Volumen o cantidad del servicio
    'TIPIFICACION',                # Clasificación adicional del servicio
    'DSTIPO_EXAMEN',
    'Siniestro_Diagnosti_Princi_Id',     # Identificador del diagnóstico/siniestro
    'Nombre_Municipio_IPS',        # Municipio descriptivo de la IPS
    'Nombre_Departamento_IPS',     # Departamento descriptivo de la IPS
    'Geogra_Municipio_Id',         # Identificador estandarizado del municipio
    'Nombre_Establecimiento',      # Establecimiento donde se prestó el servicio
    'Nombre_Municipio_Establecimiento',
    'Nombre_Departamento_Establecimiento'
]

# Leer el archivo CSV de df_salud
df_salud = pd.read_csv(
    "/kaggle/input/data-sura/1.Informacion Salud 2019-2024-001.txt",
    delimiter='|',
    usecols=cols_to_use_salud,
    dtype=dtypes_salud,
    parse_dates=['FECHA_ATENCION']
)

# Definir los tipos de dato para el dataset de prestadores
dtypes_prestadores = {
    'Geogra_Municipio_Id': 'category',
    'max_cantidad': 'float64'
}

# Seleccionar solo las columnas relevantes para df_prestadores
cols_to_use_prestadores = ['Geogra_Municipio_Id', 'max_cantidad']

# Leer el archivo CSV de df_prestadores
df_prestadores = pd.read_excel(
    "/kaggle/input/data-sura/2.Red Prestadores.xlsx",
    usecols=cols_to_use_prestadores,
    dtype=dtypes_prestadores
)

# **Analisis Exploratorio De Datos (EDA)**

In [ ]:
# Vista previa de cada dataset
print("DF Salud:")
display(df_salud.head())
print("dtypes:")
print(df_salud.dtypes)


print("DF Prestadores:")
display(df_prestadores.head())
print("dtypes:")
print(df_prestadores.dtypes)

In [ ]:
# Información general de los DataFrame
print("Información de df_salud:")
df_salud.info()

print("Informacion de Prestadores")
df_prestadores.info()

# Estadísticas descriptivas (numéricas y algunas categóricas)
print("Estadísticas descriptivas de df_salud:")
display(df_salud.describe(include='all'))

print("Estadísticas descriptivas de df_salud:")
display(df_prestadores.describe(include='all'))

**Analisis de Valores Nulos Y Duplicados**

In [ ]:
# Valores nulos
print("Valores nulos de datos de Salud")
df_salud.isnull().sum()
print("Valores nulos de datos de Prestadores")
df_prestadores.isnull().sum

In [ ]:
print("Duplicados de datos de Salud")
df_salud.duplicated()
print("DUplicados de datos de Prestadores")
df_prestadores.duplicated()

In [ ]:
# Estadísticas de capacidad máxima
display(df_prestadores["max_cantidad"].describe())

# Graficas

**Datos de Salud**

**Tipos de Atencion Medica mas Comunes**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Frecuencia de tipos de atención
plt.figure(figsize=(10, 6))
sns.countplot(data=df_salud, y="Nombre_Tipo_Atencion_Arp", order=df_salud["Nombre_Tipo_Atencion_Arp"].value_counts().index)
plt.title("Tipos de Atención Médica Más Comunes")
plt.show()

**Distribucion de categorias de CIE-10**

In [ ]:
# Agrupar códigos CIE-10 por categoría (ejemplo: primera letra)
df_salud["Categoria_CIE10"] = df_salud["Siniestro_Diagnosti_Princi_Id"].str[0]

# Gráfico de barras
plt.figure(figsize=(12, 6))
sns.countplot(data=df_salud, x="Categoria_CIE10", order=df_salud["Categoria_CIE10"].value_counts().index)
plt.title("Distribución de Categorías CIE-10")
plt.show()

In [ ]:
# Asegurarse de que FECHA_ATENCION es datetime (ya se parseó en la carga)
df_salud['Dia'] = df_salud['FECHA_ATENCION'].dt.to_period('D')

# Agrupar la demanda (Cantidad) por mes
demanda_por_mes = df_salud.groupby('Dia')['Cantidad'].sum().sort_index()

plt.figure(figsize=(32,12))
demanda_por_mes.plot()
plt.title("Demanda Mensual Total")
plt.xlabel("Dia")
plt.ylabel("Cantidad")
plt.xticks(rotation=45)
plt.show()


In [ ]:
# Agrupar y sumar las cantidades por municipio
demanda_municipio = (
    df_salud
    .groupby("Nombre_Municipio_IPS", observed=True)["Cantidad"]
    .sum()
    .reset_index()
)

# Obtener los 10 municipios con mayor demanda
top10 = demanda_municipio.nlargest(10, "Cantidad")

#print(top10)

# Ordenar los valores para el gráfico (opcional, dependiendo de la preferencia)
top10 = top10.sort_values("Cantidad", ascending=True)

# Crear el gráfico
plt.figure(figsize=(12, 6))
sns.barplot(
    data=top10,
    x="Cantidad",
    y="Nombre_Municipio_IPS",
    order=top10["Nombre_Municipio_IPS"]  # Asegura el orden correcto
)
plt.title("Municipios con Mayor Demanda de Servicios")
plt.tight_layout()  # Mejora el espacio del gráfico
plt.show()

More graohics

In [ ]:
# Distribución de Cantidad
plt.figure(figsize=(8,4))
sns.histplot(df_salud['Cantidad'], kde=True)
plt.title("Distribución de Cantidad")
plt.xlabel("Cantidad")
plt.ylabel("Frecuencia")
plt.show()

# Boxplot para detectar outliers en Cantidad
plt.figure(figsize=(8,4))
sns.boxplot(x=df_salud['Cantidad'])
plt.title("Boxplot de Cantidad")
plt.show()

# **Tendencias por tipo de servicio médico**

In [ ]:
# Asegurarse de que FECHA_ATENCION es datetime
df_salud['Mes'] = df_salud['FECHA_ATENCION'].dt.to_period('M')

# Agrupar por mes y tipo de atención
tendencia_por_tipo = df_salud.groupby(['Mes', 'Nombre_Tipo_Atencion_Arp'])['Cantidad'].sum().reset_index()

# Obtener los 5 tipos de atención más comunes para simplificar la visualización
top_tipos = df_salud['Nombre_Tipo_Atencion_Arp'].value_counts().nlargest(5).index

# Filtrar solo los tipos top
tendencia_por_tipo_top = tendencia_por_tipo[tendencia_por_tipo['Nombre_Tipo_Atencion_Arp'].isin(top_tipos)]

# Convertir Mes a string para mejor visualización
tendencia_por_tipo_top['Mes'] = tendencia_por_tipo_top['Mes'].astype(str)

# Crear gráfico de líneas
plt.figure(figsize=(15, 8))
for tipo in top_tipos:
    data = tendencia_por_tipo_top[tendencia_por_tipo_top['Nombre_Tipo_Atencion_Arp'] == tipo]
    plt.plot(data['Mes'], data['Cantidad'], marker='o', label=tipo)

plt.title('Tendencia de Demanda por Tipo de Servicio Médico')
plt.xlabel('Mes')
plt.ylabel('Cantidad')
plt.legend()
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

**Correlaciones entre variables**

In [ ]:
# Crear variables numéricas a partir de categóricas para correlación
# Codificar variables categóricas usando one-hot encoding
df_corr = pd.get_dummies(df_salud[[
    'Nombre_Tipo_Atencion_Arp', 
    'TIPIFICACION',
    'Cantidad',
    'FECHA_ATENCION'
]])

# Agregar variables temporales
df_corr['Dia_Semana'] = df_salud['FECHA_ATENCION'].dt.dayofweek
df_corr['Mes'] = df_salud['FECHA_ATENCION'].dt.month
df_corr['Año'] = df_salud['FECHA_ATENCION'].dt.year

# Calcular matriz de correlación
corr_matrix = df_corr.corr()

# Visualizar matriz de correlación
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=False, cmap='coolwarm', center=0)
plt.title('Matriz de Correlación')
plt.tight_layout()
plt.show()

**Análisis específico por municipio**

In [ ]:
# Agrupar datos por municipio
datos_por_municipio = df_salud.groupby('Nombre_Municipio_IPS').agg({
    'Cantidad': 'sum',
    'FECHA_ATENCION': 'count',
    'Nombre_Tipo_Atencion_Arp': lambda x: x.nunique(),
    'TIPIFICACION': lambda x: x.nunique()
}).reset_index()

# Renombrar columnas
datos_por_municipio.columns = ['Municipio', 'Total_Demanda', 'Num_Atenciones', 'Tipos_Servicio', 'Tipificaciones']

# Calcular promedio de demanda por atención
datos_por_municipio['Demanda_Promedio'] = datos_por_municipio['Total_Demanda'] / datos_por_municipio['Num_Atenciones']

# Visualizar estadísticas por municipio (todos los municipios)
print("Estadísticas por municipio:")
display(datos_por_municipio.sort_values('Total_Demanda', ascending=False))

# Analizar más a fondo los 10 municipios principales
top10_municipios = datos_por_municipio.nlargest(10, 'Total_Demanda')['Municipio'].tolist()

# Para cada municipio top, analizar la tendencia mensual
plt.figure(figsize=(15, 10))

for i, municipio in enumerate(top10_municipios):
    # Filtrar datos del municipio
    df_muni = df_salud[df_salud['Nombre_Municipio_IPS'] == municipio]
    
    # Agrupar por mes
    df_muni_mes = df_muni.groupby(df_muni['FECHA_ATENCION'].dt.to_period('M'))['Cantidad'].sum()
    
    # Crear subgráfico
    plt.subplot(5, 2, i+1)
    df_muni_mes.plot()
    plt.title(f'Tendencia mensual: {municipio}')
    plt.xlabel('Mes')
    plt.ylabel('Cantidad')
    plt.grid(True)

plt.tight_layout()
plt.show()

**Patrones estacionales**

In [ ]:
# Crear variables para diferentes medidas temporales
df_salud['Año'] = df_salud['FECHA_ATENCION'].dt.year
df_salud['Mes'] = df_salud['FECHA_ATENCION'].dt.month
df_salud['DiaSemana'] = df_salud['FECHA_ATENCION'].dt.dayofweek  # 0=Lunes, 6=Domingo
df_salud['DiaMes'] = df_salud['FECHA_ATENCION'].dt.day
df_salud['Trimestre'] = df_salud['FECHA_ATENCION'].dt.quarter

# 1. Análisis por mes del año
demanda_por_mes = df_salud.groupby('Mes')['Cantidad'].sum().reset_index()

plt.figure(figsize=(10, 6))
sns.barplot(x='Mes', y='Cantidad', data=demanda_por_mes)
plt.title('Demanda por Mes del Año')
plt.xlabel('Mes')
plt.ylabel('Cantidad')
plt.xticks(range(12), ['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic'])
plt.show()

# 2. Análisis por día de la semana
demanda_por_dia = df_salud.groupby('DiaSemana')['Cantidad'].sum().reset_index()

plt.figure(figsize=(10, 6))
sns.barplot(x='DiaSemana', y='Cantidad', data=demanda_por_dia)
plt.title('Demanda por Día de la Semana')
plt.xlabel('Día de la Semana')
plt.ylabel('Cantidad')
plt.xticks(range(7), ['Lun', 'Mar', 'Mié', 'Jue', 'Vie', 'Sáb', 'Dom'])
plt.show()

# 3. Análisis por trimestre
demanda_por_trimestre = df_salud.groupby(['Año', 'Trimestre'])['Cantidad'].sum().reset_index()

plt.figure(figsize=(12, 6))
sns.lineplot(x='Trimestre', y='Cantidad', hue='Año', data=demanda_por_trimestre, marker='o')
plt.title('Demanda Trimestral por Año')
plt.xlabel('Trimestre')
plt.ylabel('Cantidad')
plt.grid(True)
plt.show()

# 4. Análisis de estacionalidad avanzado (descomposición)
from statsmodels.tsa.seasonal import seasonal_decompose

# Preparar serie temporal para decomposición
ts_data = df_salud.groupby(df_salud['FECHA_ATENCION'].dt.to_period('M'))['Cantidad'].sum()
ts_data.index = pd.to_datetime(ts_data.index.astype(str))

# Realizar descomposición estacional (si hay suficientes datos)
if len(ts_data) >= 24:  # Necesitamos al menos 2 años de datos
    decomposition = seasonal_decompose(ts_data, model='additive', period=12)
    
    # Graficar la descomposición
    plt.figure(figsize=(12, 10))
    plt.subplot(411)
    plt.plot(ts_data, label='Original')
    plt.legend(loc='best')
    plt.title('Descomposición Estacional')
    
    plt.subplot(412)
    plt.plot(decomposition.trend, label='Tendencia')
    plt.legend(loc='best')
    
    plt.subplot(413)
    plt.plot(decomposition.seasonal, label='Estacionalidad')
    plt.legend(loc='best')
    
    plt.subplot(414)
    plt.plot(decomposition.resid, label='Residuos')
    plt.legend(loc='best')
    
    plt.tight_layout()
    plt.show()

**Detección de valores atípicos, datos faltantes y errores**

In [ ]:
# 1. Análisis de valores faltantes
print("Total de valores faltantes por columna:")
print(df_salud.isnull().sum())

# Visualización de valores faltantes
plt.figure(figsize=(12, 6))
sns.heatmap(df_salud.isnull(), cbar=False, yticklabels=False, cmap='viridis')
plt.title('Mapa de Valores Faltantes')
plt.show()

# 2. Detección de valores atípicos en la variable "Cantidad"
plt.figure(figsize=(10, 6))
sns.boxplot(y=df_salud['Cantidad'])
plt.title('Distribución de la Variable Cantidad')
plt.ylabel('Cantidad')
plt.show()

# Cálculo de percentiles para identificar outliers
Q1 = df_salud['Cantidad'].quantile(0.25)
Q3 = df_salud['Cantidad'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f"Valores atípicos - Límite inferior: {lower_bound}, Límite superior: {upper_bound}")
print(f"Número de outliers: {df_salud[(df_salud['Cantidad'] < lower_bound) | (df_salud['Cantidad'] > upper_bound)].shape[0]}")

# 3. Análisis de valores inconsistentes
# Verificar si hay cantidades negativas o cero (que podrían ser erróneas)
print(f"Registros con cantidad negativa: {df_salud[df_salud['Cantidad'] < 0].shape[0]}")
print(f"Registros con cantidad cero: {df_salud[df_salud['Cantidad'] == 0].shape[0]}")

# 4. Verificación de fechas
min_date = df_salud['FECHA_ATENCION'].min()
max_date = df_salud['FECHA_ATENCION'].max()
print(f"Rango de fechas: {min_date} a {max_date}")

# Comprobar si hay fechas futuras (posteriores a la fecha actual)
future_dates = df_salud[df_salud['FECHA_ATENCION'] > pd.Timestamp.now()]
print(f"Registros con fechas futuras: {future_dates.shape[0]}")

# 5. Verificar duplicados exactos
duplicados = df_salud.duplicated()
print(f"Número de registros duplicados: {duplicados.sum()}")

# 6. Comprobar problemas potenciales en los valores categóricos
for col in ['Nombre_Tipo_Atencion_Arp', 'TIPIFICACION', 'Nombre_Municipio_IPS']:
    n_values = df_salud[col].nunique()
    print(f"Columna {col}: {n_values} valores únicos")
    
    # Si hay muchos valores únicos, puede haber inconsistencias
    if n_values > 100:  # Umbral arbitrario
        print("  ⚠️ Esta columna tiene muchos valores únicos, posible inconsistencia en nomenclatura")
        
        # Mostrar los valores más y menos frecuentes
        print("  Valores más comunes:")
        print(df_salud[col].value_counts().head())
        print("  Valores menos comunes:")
        print(df_salud[col].value_counts().tail())

# 7. Análisis de la capacidad máxima del prestador vs. demanda real
if 'Geogra_Municipio_Id' in df_salud.columns and 'Geogra_Municipio_Id' in df_prestadores.columns:
    # Agrupar demanda por municipio
    demanda_municipio = df_salud.groupby('Geogra_Municipio_Id')['Cantidad'].sum().reset_index()
    
    # Combinar con información de capacidad
    analisis_capacidad = demanda_municipio.merge(df_prestadores, on='Geogra_Municipio_Id', how='inner')
    analisis_capacidad['Porcentaje_Uso'] = (analisis_capacidad['Cantidad'] / analisis_capacidad['max_cantidad']) * 100
    
    # Identificar posibles problemas
    print("Municipios con posible sobrecarga (uso > 90%):")
    display(analisis_capacidad[analisis_capacidad['Porcentaje_Uso'] > 90])
    
    print("Municipios con baja utilización (uso < 20%):")
    display(analisis_capacidad[analisis_capacidad['Porcentaje_Uso'] < 20])

**Análisis de series temporales específico**

In [ ]:
import matplotlib.dates as mdates
from statsmodels.tsa.stattools import adfuller, acf, pacf
import statsmodels.api as sm

# Preparar serie temporal diaria
ts_diaria = df_salud.groupby(df_salud['FECHA_ATENCION'].dt.date)['Cantidad'].sum()
ts_diaria.index = pd.to_datetime(ts_diaria.index)

# 1. Visualizar la serie temporal completa
plt.figure(figsize=(15, 7))
plt.plot(ts_diaria)
plt.title('Serie Temporal Diaria de Demanda')
plt.xlabel('Fecha')
plt.ylabel('Cantidad')
plt.grid(True)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=3))
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# 2. Analizar la estacionariedad de la serie
result = adfuller(ts_diaria.dropna())
print('Prueba de Dickey-Fuller Aumentada:')
print(f'Estadística de prueba: {result[0]}')
print(f'Valor p: {result[1]}')
print(f'Valores críticos: {result[4]}')
if result[1] <= 0.05:
    print("La serie es estacionaria (rechazamos H0)")
else:
    print("La serie no es estacionaria (no rechazamos H0)")

# 3. Analizar autocorrelación
plt.figure(figsize=(12, 10))

# ACF
plt.subplot(211)
sm.graphics.tsa.plot_acf(ts_diaria.dropna(), lags=40, ax=plt.gca())
plt.title('Función de Autocorrelación (ACF)')

# PACF
plt.subplot(212)
sm.graphics.tsa.plot_pacf(ts_diaria.dropna(), lags=40, ax=plt.gca())
plt.title('Función de Autocorrelación Parcial (PACF)')

plt.tight_layout()
plt.show()

# 4. Descomposición de la serie temporal (tendencia, estacionalidad, residuos)
# Usar la serie semanal para reducir el ruido
ts_semanal = df_salud.groupby(pd.Grouper(key='FECHA_ATENCION', freq='W'))['Cantidad'].sum()

# Realizar descomposición
descomposicion = sm.tsa.seasonal_decompose(ts_semanal, model='additive', period=52)

# Graficar
fig = descomposicion.plot()
fig.set_size_inches(15, 12)
plt.tight_layout()
plt.show()

# 5. Análisis de venta móvil para detectar tendencias a largo plazo
ventanas = [7, 30, 90, 180]
plt.figure(figsize=(15, 8))

plt.plot(ts_diaria, alpha=0.5, label='Original')

for ventana in ventanas:
    ts_ma = ts_diaria.rolling(window=ventana).mean()
    plt.plot(ts_ma, label=f'Media Móvil ({ventana} días)')

plt.title('Análisis de Tendencia con Medias Móviles')
plt.xlabel('Fecha')
plt.ylabel('Cantidad')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# **Limpieza de Datos**

In [ ]:
from datetime import datetime, timedelta

# Assuming df_salud and df_prestadores are already loaded

# 1. Handle duplicates
print(f"Shape before removing duplicates: {df_salud.shape}")
df_salud = df_salud.drop_duplicates()
print(f"Shape after removing duplicates: {df_salud.shape}")

# 2. Clean location data
# Fix "?" values in municipios - replace with NaN for now
df_salud['Nombre_Municipio_IPS'] = df_salud['Nombre_Municipio_IPS'].replace({'?', 'Nombre_Municipio_Ips', 'Nombre_Municipio_IPS'}, np.nan)

# Standardize municipality names (capitalize properly)
df_salud['Nombre_Municipio_IPS'] = df_salud['Nombre_Municipio_IPS'].str.title()
df_salud['Nombre_Departamento_IPS'] = df_salud['Nombre_Departamento_IPS'].str.title()

# 3. Handle outliers in 'Cantidad'
# Calculate IQR
Q1 = df_salud['Cantidad'].quantile(0.01)
Q3 = df_salud['Cantidad'].quantile(0.99)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f"Outlier bounds - Lower: {lower_bound}, Upper: {upper_bound}")
print(f"Outliers count: {df_salud[(df_salud['Cantidad'] < lower_bound) | (df_salud['Cantidad'] > upper_bound)].shape[0]}")

# Create a copy of the dataframe with outliers capped/floored
df_salud_cleaned = df_salud.copy()

# Option 1: Cap outliers (winsorizing)
df_salud_cleaned['Cantidad_capped'] = np.where(
    df_salud_cleaned['Cantidad'] > upper_bound,
    upper_bound,
    np.where(
        df_salud_cleaned['Cantidad'] < lower_bound,
        lower_bound,
        df_salud_cleaned['Cantidad']
    )
)

# 4. Fill missing municipality data where possible
# If we have municipality ID but no name, we could try to fill it from other records
municipality_mapping = df_salud[['Geogra_Municipio_Id', 'Nombre_Municipio_IPS']].dropna().drop_duplicates()
municipality_mapping = municipality_mapping.groupby('Geogra_Municipio_Id')['Nombre_Municipio_IPS'].first().reset_index()

# Create a dictionary for mapping
muni_id_to_name = dict(zip(municipality_mapping['Geogra_Municipio_Id'], municipality_mapping['Nombre_Municipio_IPS']))

# Fill missing municipality names using the ID mapping
missing_muni_mask = df_salud_cleaned['Nombre_Municipio_IPS'].isna() & df_salud_cleaned['Geogra_Municipio_Id'].notna()
df_salud_cleaned.loc[missing_muni_mask, 'Nombre_Municipio_IPS'] = \
    df_salud_cleaned.loc[missing_muni_mask, 'Geogra_Municipio_Id'].map(muni_id_to_name)

# 4. Fill missing municipality data where possible
# If we have municipality ID but no name, we could try to fill it from other records
municipality_mapping = df_salud[['Geogra_Municipio_Id', 'Nombre_Municipio_IPS']].dropna().drop_duplicates()
municipality_mapping = municipality_mapping.groupby('Geogra_Municipio_Id')['Nombre_Municipio_IPS'].first().reset_index()

# Create a dictionary for mapping
muni_id_to_name = dict(zip(municipality_mapping['Geogra_Municipio_Id'], municipality_mapping['Nombre_Municipio_IPS']))

# Fill missing municipality names using the ID mapping
missing_muni_mask = df_salud_cleaned['Nombre_Municipio_IPS'].isna() & df_salud_cleaned['Geogra_Municipio_Id'].notna()
df_salud_cleaned.loc[missing_muni_mask, 'Nombre_Municipio_IPS'] = \
    df_salud_cleaned.loc[missing_muni_mask, 'Geogra_Municipio_Id'].map(muni_id_to_name)

# 6. Data consistency check after cleaning
print("\nData summary after cleaning:")
print(f"Total records: {df_salud_cleaned.shape[0]}")
print(f"Missing municipality names: {df_salud_cleaned['Nombre_Municipio_IPS'].isna().sum()}")
print(f"Date range: {df_salud_cleaned['FECHA_ATENCION'].min()} to {df_salud_cleaned['FECHA_ATENCION'].max()}")
print(f"Date range: {df_salud_cleaned['Cantidad_capped'].min()} to {df_salud_cleaned['Cantidad_capped'].max()}")